<a href="https://www.kaggle.com/code/hetarthchopra/sentence-transformer-nli?scriptVersionId=112834516" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv
/kaggle/input/sentence-transformers-dataset/__results__.html
/kaggle/input/sentence-transformers-dataset/__notebook__.ipynb
/kaggle/input/sentence-transformers-dataset/__output__.json
/kaggle/input/sentence-transformers-dataset/custom.css
/kaggle/input/sentence-transformers-dataset/sentence-transformers/index.rst
/kaggle/input/sentence-transformers-dataset/sentence-transformers/NOTICE.txt
/kaggle/input/sentence-transformers-dataset/sentence-transformers/setup.cfg
/kaggle/input/sentence-transformers-dataset/sentence-transformers/LICENSE
/kaggle/input/sentence-transformers-dataset/sentence-transformers/.gitignore
/kaggle/input/sentence-transformers-dataset/sentence-transformers/README.md
/kaggle/input/sentence-transformers-dataset/sentence-transformers/requirements.txt
/kaggle/input/sentence-transfor

Thanks to the work of [here](https://www.kaggle.com/code/tanlikesmath/pretrained-sentence-transformer-model-baseline), for uploading sentence transformers library and models on kaggle. 

In [2]:
!cp -r ../input/sentence-transformers-dataset/sentence-transformers /tmp/st
!pip install /tmp/st

Processing /tmp/st
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=126135 sha256=b0d2723b0ac1e54b80bc1ab2399a5942f6e9846c9de8ee84839ca06e2ca09381
  Stored in directory: /tmp/pip-ephem-wheel-cache-h7kypwgv/wheels/07/90/8c/ea25d1a415b340d7b89c6ceb0aec3184514860bcf8813763ae
Successfully built sentence-transformers


# Data Preprocessing and Cleaning

In [3]:
train = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
train = train.head(150)
cpc_codes = pd.read_csv('/kaggle/input/cpc-codes/titles.csv')
cpc_codes.head()

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0


## Check if data is missing or not

In [4]:
list_of_cpc_codes_in_train = train.context.unique().tolist()
print(len(list_of_cpc_codes_in_train))
all_codes_list  = cpc_codes.code.unique().tolist()
print(len(all_codes_list))
codes_list = [i for i in all_codes_list if i in list_of_cpc_codes_in_train] 
print(codes_list)
# filter cpc codes that are in the codes list
cpc_codes_train = cpc_codes[cpc_codes['code'].isin(codes_list)]

14
260476
['A47', 'A61', 'A62', 'B23', 'B41', 'C01', 'C08', 'D03', 'E03', 'F16', 'F24', 'F28', 'H01', 'H04']


## Replace codes with the category names

In [5]:
train['context'] = train['context'].apply(lambda x: cpc_codes_train.loc[cpc_codes_train['code'] == str(x)]['title'].values[0])
train['appender'] = train['anchor'] + ' ' + train['context']

In [6]:
train.score.value_counts()

0.25    56
0.50    47
0.00    33
0.75    13
1.00     1
Name: score, dtype: int64

In [7]:
train

,id,anchor,target,context,score,appender
0,37d61fd2272659b1,abatement,abatement of pollution,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.50,abatement FURNITURE; DOMESTIC ARTICLES OR APPL...
1,7b9652b17b68b7a4,abatement,act of abating,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.75,abatement FURNITURE; DOMESTIC ARTICLES OR APPL...
2,36d72442aefd8232,abatement,active catalyst,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.25,abatement FURNITURE; DOMESTIC ARTICLES OR APPL...
3,5296b0c19e1ce60e,abatement,eliminating process,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.50,abatement FURNITURE; DOMESTIC ARTICLES OR APPL...
4,54c1e3b9184cb5b6,abatement,forest region,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.00,abatement FURNITURE; DOMESTIC ARTICLES OR APPL...
...,...,...,...,...,...,...
145,16479a99b5c37673,absorbent properties,ionic compound,ORGANIC MACROMOLECULAR COMPOUNDS; THEIR PREPAR...,0.25,absorbent properties ORGANIC MACROMOLECULAR CO...
146,ec050b5360ebba22,absorbent properties,liquid holding capacity,ORGANIC MACROMOLECULAR COMPOUNDS; THEIR PREPAR...,0.50,absorbent properties ORGANIC MACROMOLECULAR CO...
147,0eba34ed68ed88b8,absorbent properties,mechanical characteristics,ORGANIC MACROMOLECULAR COMPOUNDS; THEIR PREPAR...,0.25,absorbent properties ORGANIC MACROMOLECULAR CO...
148,ef8a6631ccd28df0,absorbent properties,mechanical properties,ORGANIC MACROMOLECULAR COMPOUNDS; THEIR PREPAR...,0.25,absorbent properties ORGANIC MACROMOLECULAR CO...


# Defining and Training the Model

In [8]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util, SentencesDataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
# for cross encoder
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator,CECorrelationEvaluator
from sentence_transformers.readers import InputExample
from datetime import datetime
from sklearn.model_selection import train_test_split

In [9]:
train, test = train_test_split(train, test_size=0.2)
train_samples = []
test_samples = []
test_samples_list = []
label2int = {0.0: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4}

# define the train dataset
for index, row in train.iterrows():
#     label_id = label2int[row['score']]
    inp_example = InputExample(texts=[row['target'], row['appender']], label=row['score'])
    train_samples.append(inp_example)

# define the test dataset
for index, row in test.iterrows():
#     label_id = label2int[row['score']]
    inp_example = InputExample(texts=[row['target'], row['appender']], label=row['score'])
    inp_example_text = [row['target'], row['appender']]
    test_samples.append(inp_example)
    test_samples_list.append(inp_example_text)    

In [10]:
model_name = '../input/sentence-transformers-models/all-MiniLM-L6-v2'

# Read the dataset
train_batch_size = 128
# number of epochs
num_epochs = 20

# softmax model
model = CrossEncoder(model_name, num_labels=len(label2int))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../input/sentence-transformers-models/all-MiniLM-L6-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(test_samples, name='AllNLI-dev')
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.01)

In [12]:
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path='/kaggle/working/model')

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(test_samples, name='sts-test')
evaluator(model)

0.26666666666666666

## Prediction of the test samples

In [14]:
validation = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')


In [15]:
validation['context'] = validation['context'].apply(lambda x: cpc_codes.loc[cpc_codes['code'] == str(x)]['title'].values[0])
validation['appender'] = validation['anchor'] + ' ' + validation['context']

In [16]:
validation_samples_list = []
for index, row in validation.iterrows():
    inp_example_text = [row['target'], row['appender']]
    validation_samples_list.append(inp_example_text)    

In [17]:
predictions = model.predict(validation_samples_list)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
output_labels = []
for i in predictions:
    output_labels.append(np.argmax(i))

# output_labels_after_dict = []
# for i in output_labels:
#     output_labels_after_dict.append(float([k for k,v in label2int.items() if v == i][0]))

In [19]:
# validation['score'] = output_labels_after_dict

In [20]:
validation = validation.drop(columns = ['anchor','target','context','appender'])

In [21]:
validation.to_csv('submission.csv' , index=False)

In [22]:
validation

,id
0,4112d61851461f60
1,09e418c93a776564
2,36baf228038e314b
3,1f37ead645e7f0c8
4,71a5b6ad068d531f
5,474c874d0c07bd21
6,442c114ed5c4e3c9
7,b8ae62ea5e1d8bdb
8,faaddaf8fcba8a3f
9,ae0262c02566d2ce
